<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Replace MongoDB® and Pinecone with SingleStore</h1>
    </div>
</div>

In [7]:
!pip cache purge --quiet

In [8]:
!pip install openai --quiet
!pip install pinecone --quiet

In [9]:
import numpy as np
import os
import pandas as pd
import time

from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from pymongo import MongoClient
from singlestoredb.management import get_secret

In [10]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")
pc_api_key = get_secret("PINECONE_API_KEY")

In [11]:
dimensions = 1536

## Pre-AI Email Marketing - "Spray and Pray"

We've all received these forms of emails where it's clear that they just send out the same email to every person in their email list, rather than actually looking at what kind of customer we are.

Sometimes it's even worse than just using our first name on the preamble. I'm sure you've received emails like this before too:

```
Dear $fname,

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras semper at urna sit amet imperdiet. Integer rutrum tempor tellus et porta. Mauris elit dui, euismod a ipsum sit amet, ultricies dictum ex. Suspendisse bibendum suscipit leo sed egestas. Suspendisse potenti. Maecenas placerat lectus eget tincidunt posuere. Ut aliquet elit ante. Duis vel volutpat massa, pulvinar condimentum enim. Aliquam erat volutpat. Sed vulputate pretium mauris, eu tempus arcu lacinia sed. Aliquam lobortis massa at ligula aliquet tincidunt. Aliquam ultrices sem a ultricies pulvinar. In facilisis pretium cursus. Vestibulum et malesuada justo. Vivamus erat ex, aliquet nec porttitor eu, sodales consectetur dolor.

Sincerely,
Marketing Person
@ Company
```

Below is a simplistic example of creating that email:

In [12]:
people = ["Alice", "Bob", "Charlie", "David", "Emma"]

for person in people:
    message = (
        f"Hey {person},\n"
        "Check out our web analytics platform, it's Awesome!\n"
        "It's perfect for your needs. Buy it now!\n"
        "- Marketer John"
    )
    print(message)
    print("_" * 100)

Hey Alice,
Check out our web analytics platform, it's Awesome!
It's perfect for your needs. Buy it now!
- Marketer John
____________________________________________________________________________________________________
Hey Bob,
Check out our web analytics platform, it's Awesome!
It's perfect for your needs. Buy it now!
- Marketer John
____________________________________________________________________________________________________
Hey Charlie,
Check out our web analytics platform, it's Awesome!
It's perfect for your needs. Buy it now!
- Marketer John
____________________________________________________________________________________________________
Hey David,
Check out our web analytics platform, it's Awesome!
It's perfect for your needs. Buy it now!
- Marketer John
____________________________________________________________________________________________________
Hey Emma,
Check out our web analytics platform, it's Awesome!
It's perfect for your needs. Buy it now!
- Marketer Jo

## A Large Language Model (LLM) has entered the chat

If there's something that will help us stand out as a marketer, it's when we send emails that are roughly the same in content but we use an LLM to generate the actual content. Below we'll see how we could prompt OpenAI for some "unique" emails to send out to people.

### Setup OpenAI

The system prompt helps prime ChatGPT to respond with content that is helpful for us. Notice that we explicity request for ChatGPT to lookup the top 5 web analytics companies in order to give it a small corpus of content to work from for this example.

In [13]:
system_message = """
You are a helpful assistant.
My name is Marketer John.
You help write the body of an email for a fictitious company called 'Awesome Web Analytics'.
This is a web analytics company that is similar to the top 5 web analytics companies (perform a web search to determine the current top 5 web analytics companies).
The goal is to write a custom email to users to get them interested in our services.
The email should be less than 150 words.
Address the user by name.
End with my signature.
"""

### Loop through and request

Now we'll loop through each person and generate the emails by querying ChatGPT, and storing the responses in a Python list.

In [14]:
def chatgpt_generate_email(prompt, person):
    conversation = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": person},
        {"role": "assistant", "content": ""}
    ]

    response = openai_client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = conversation,
        temperature = 1.0,
        max_tokens = 800,
        top_p = 1,
        frequency_penalty = 0,
        presence_penalty = 0
    )

    assistant_reply = response.choices[0].message.content
    return assistant_reply

In [15]:
openai_client = OpenAI()

# Define a list to store the responses
emails = []

# Loop through each person and generate the conversation
for person in people:
    email = chatgpt_generate_email(system_message, person)
    emails.append(
        {
            "person": person,
            "assistant_reply": email
        }
    )

### Print out the list

Now we print out the emails, with names and everything. Note that each of the responses is unique and completely random.

In [19]:
for email in emails:
    person = email["person"]
    assistant_reply = email["assistant_reply"]
    print(f"Person: {person}")
    print(f"{assistant_reply}")
    print("_" * 100)

Person: Alice
Subject: Unlock the Power of Your Web Data with Awesome Web Analytics!

Hi Alice,

Are you ready to take your website's performance to the next level? At Awesome Web Analytics, we provide cutting-edge analytics solutions designed to give you deep insights into your web traffic. 

Unlike traditional competitors, our user-friendly platform allows you to visualize trends, track user behavior, and optimize conversions effortlessly. With real-time data and actionable insights, you'll have the tools to make informed decisions that drive success.

Join the ranks of successful businesses harnessing the power of their web analytics. Discover how we can help you understand your audience better and boost your digital strategy!

Best regards,

Marketer John  
Awesome Web Analytics  

____________________________________________________________________________________________________
Person: Bob
Subject: Unlock Your Website's Potential with Awesome Web Analytics!

Hi Bob,

Are you rea

## Customising the content to user behaviour

Enhancing our performance can be achieved readily when we have access to customer behaviour data within our application. For example, what if a user had registered for our service but had not yet completed the installation of the analytics tracking code on their website?

We have a MongoDB database populated with some user data that looks like this:

```
{
    '_id': ObjectId('64afb3fda9295d8421e7a19f'),
    'first_name': 'James',
    'last_name': 'Villanueva',
    'company_name': 'Foley-Turner',
    'stage': 'generating a tracking code',
    'created_date': 1987-11-09T12:43:26.000+00:00
}
```

Take particular note of the key `stage`, we will be using that to help identify user behaviour and have ChatGPT enhance our emails.

Below we use MongoDB as a source for user behaviour, then use that to have ChatGPT write some more useful emails for the users.

### Setup MongoDB Atlas

We’ll use MongoDB Atlas in an M0 Sandbox. We'll configure an **admin** user with **atlasAdmin** privileges under **Database Access**. We’ll temporarily allow access from anywhere (IP Address 0.0.0.0/0) under **Network Access**. We’ll note down the `password` and `host`.

### Load the data into MongoDB Atlas

Next, we'll load the sample data into MongoDB Atlas. We’ll create a new database called `mktg_email_demo` with a collection called `customers`. We’ll load data into this collection. The data load can be accomplished in several ways, such as using MongoDB Compass or mongoimport. The dataset can be found on [GitHub](https://github.com/VeryFatBoy/mktg-email-flow).

### Connect to MongoDB Atlas

Below, we'll connect to MongoDB Atlas. We’ll replace `<password>` and `<host>` with the values that we saved earlier from MongoDB Atlas.

In [20]:
try:
    mongo_client = MongoClient("mongodb+srv://admin:<password>@<host>/?retryWrites=true&w=majority")
    mongo_db = mongo_client["mktg_email_demo"]
    collection = mongo_db["customers"]
    print("Connected successfully")
except Exception as e:
    print(e)

Connected successfully


In [21]:
stages = [
    "getting started",
    "generating a tracking code",
    "adding tracking to your website",
    "real-time analytics",
    "conversion tracking",
    "funnels",
    "user segmentation",
    "custom event tracking",
    "data export",
    "dashboard customization"
]

def find_next_stage(current_stage):
    current_index = stages.index(current_stage)
    if current_index < len(stages) - 1:
        return stages[current_index + 1]
    else:
        return stages[current_index]

### Loop through the users in the collection

Limiting to 5 users, we'll loop through each record in our customers collection in MongoDB Atlas.

In [22]:
limit = 5
emails = []

for record in collection.find(limit = limit):
    fname, stage = record.get("first_name"), record.get("stage")
    next_stage = find_next_stage(stage)

    system_message = f"""
    You are a helpful assistant, who works for me, Marketer John at Awesome Web Analytics.
    You help write the body of an email for a fictitious company called 'Awesome Web Analytics'.
    We are a web analytics company similar to the top 5 web analytics companies.
    We have users at various stages in our product's pipeline, and we want to send them helpful emails to encourage further usage of our product.
    Please write an email for {fname} who is on stage {stage} of the onboarding process.
    The next stage is {next_stage}.
    Ensure the email describes the benefits of moving to the next stage.
    Limit the email to 1 paragraph.
    End the email with my signature.
    """

    email = chatgpt_generate_email(system_message, fname)
    emails.append(
        {
            "fname": fname,
            "stage": stage,
            "next_stage": next_stage,
            "email": email
        }
    )

### Print out the list

Now let's take a look at what those emails look like now.

In [23]:
for email in emails:
    print(f"First Name: {email['fname']}\n")
    print(f"Stage: {email['stage']}\n")
    print(f"Next Stage: {email['next_stage']}\n")
    print(f"{email['email']}\n")
    print("_" * 100)

First Name: Michael

Stage: funnels

Next Stage: user segmentation

Subject: Unlock the Power of User Segmentation!

Hi Michael,

Congratulations on successfully navigating the funnels stage of your onboarding process! Moving on to user segmentation will allow you to dive deeper into your analytics and gain invaluable insights about your audience. By grouping users based on their behaviors, preferences, and demographics, you'll be able to tailor your marketing strategies more effectively, enhance user engagement, and ultimately drive conversions. This next step is crucial for maximizing the value you get from Awesome Web Analytics, and we're here to support you every step of the way!

Best regards,  
Marketer John  
Awesome Web Analytics

____________________________________________________________________________________________________
First Name: Robert

Stage: funnels

Next Stage: user segmentation

Subject: Unlock the Power of User Segmentation

Hi Robert,

Congratulations on reac

## What about adding in vectors?

It would be useful to be able to drive users to documentation to ensure that they are able to complete what they're working on. So let's use a Vector Database (Pinecone) then perform a query against it to determine the best place in our documentation to send our users.

In [25]:
pc = Pinecone(
    api_key = pc_api_key
)

index_name = "mktg-email-demo"

if any(index["name"] == index_name for index in pc.list_indexes()):
    pc.delete_index(index_name)

pc.create_index(
    name = index_name,
    dimension = dimensions,
    metric = "euclidean",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)

pc_index = pc.Index(index_name)

pc.list_indexes()

[
    {
        "name": "mktg-email-demo",
        "dimension": 1536,
        "metric": "euclidean",
        "host": "mktg-email-demo-j7nv9g6.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

### Create embeddings for our stages

We need embeddings for each of the stages in our pipeline, so we can perform searches in Pinecone for the best docs.

In [26]:
def get_embeddings(text):
    text = text.replace("\n", " ")
    try:
        response = openai_client.embeddings.create(
            input = text,
            model = "text-embedding-3-small"
        )
        return response.data[0].embedding, response.usage.total_tokens, "success"
    except Exception as e:
        print(e)
        return "", 0, "failed"

In [27]:
stages_w_embed = []

for stage in stages:
    embedding, tokens, status = get_embeddings(stage)
    stages_w_embed.append({"stage": stage, "embedding": embedding})

In [28]:
id_counter = 1
ids_list = []

for stage in stages:
    embedding, tokens, status = get_embeddings(stage)

    parent = id_counter - 1

    pc_index.upsert([
        {
            "id": str(id_counter),
            "values": embedding,
            "metadata": {"content": stage, "parent": str(parent)}
        }
    ])

    ids_list.append(str(id_counter))

    id_counter += 1

In [29]:
time.sleep(15)

response = pc_index.fetch(ids = ids_list)

for vector_id, vector_data in response["vectors"].items():
    print(vector_data["metadata"])

{'content': 'dashboard customization', 'parent': '9'}
{'content': 'generating a tracking code', 'parent': '1'}
{'content': 'funnels', 'parent': '5'}
{'content': 'real-time analytics', 'parent': '3'}
{'content': 'user segmentation', 'parent': '6'}
{'content': 'custom event tracking', 'parent': '7'}
{'content': 'conversion tracking', 'parent': '4'}
{'content': 'getting started', 'parent': '0'}
{'content': 'data export', 'parent': '8'}
{'content': 'adding tracking to your website', 'parent': '2'}


### Search Pinecone for closest result

Using the embeddings for each stage, generated in the last step, we can search the Pinecone index for the closest matching document in our docs.

In [30]:
def search_pinecone(embedding):
    match = pc_index.query(
        vector = [embedding],
        top_k = 1,
        include_metadata = True
    )["matches"][0]["metadata"]
    return match["content"], match["parent"]

### Loop through the users in the collection

Limiting to 5 users, we'll loop through each record in our customers collection in MongoDB Atlas.

In [31]:
limit = 5
emails = []

for record in collection.find(limit = limit):
    fname, stage = record.get("first_name"), record.get("stage")

    # Get the current and next stages with their embedding
    this_stage = next((item for item in stages_w_embed if item["stage"] == stage), None)
    next_stage = next((item for item in stages_w_embed if item["stage"] == find_next_stage(stage)), None)

    if not this_stage or not next_stage:
        continue

    # Get content
    cur_content, cur_permalink = search_pinecone(this_stage["embedding"])
    next_content, next_permalink = search_pinecone(next_stage["embedding"])

    system_message = f"""
    You are a helpful assistant.
    I am Marketer John at Awesome Web Analytics.
    We are similar to the current top web analytics companies.
    We have users at various stages of using our product, and we want to send them helpful emails to encourage them to use our product more.
    Write an email for {fname}, who is on stage {stage} of the onboarding process.
    The next stage is {next_stage['stage']}.
    Ensure the email describes the benefits of moving to the next stage, and include this link: https://github.com/VeryFatBoy/mktg-email-flow/tree/main/docs/{next_content.replace(' ', '-')}.md.
    Limit the email to 1 paragraph.
    End the email with my signature: 'Best Regards, Marketer John.'
    """

    email = chatgpt_generate_email(system_message, fname)
    emails.append(
        {
            "fname": fname, 
            "stage": stage, 
            "next_stage": next_stage["stage"], 
            "email": email
        }
    )

### Print out the list

Now, let's print out the emails.

In [32]:
for email in emails:
    print(f"First Name: {email['fname']}\n")
    print(f"Stage: {email['stage']}\n")
    print(f"Next Stage: {email['next_stage']}\n")
    print(f"{email['email']}\n")
    print("_" * 100)

First Name: Michael

Stage: funnels

Next Stage: user segmentation

Subject: Unlock the Power of User Segmentation!

Hi Michael,

Congratulations on reaching the funnels stage of our onboarding process! To fully leverage the capabilities of Awesome Web Analytics, the next step is user segmentation, which allows you to target specific audiences more effectively, understand their behaviors, and tailor your marketing strategies for better results. This will enable you to gain deeper insights into your users and increase the overall efficiency of your campaigns. You can learn more about the benefits and implementation of user segmentation here: [User Segmentation Guide](https://github.com/VeryFatBoy/mktg-email-flow/tree/main/docs/user-segmentation.md). 

Best Regards,  
Marketer John.

____________________________________________________________________________________________________
First Name: Robert

Stage: funnels

Next Stage: user segmentation

Subject: Unlock the Power of User Segme

## Let's make this easier with SingleStore

### Ingest data live into SingleStore

Before we go on to the steps of querying our data and simplifying the process down to one database, let's ingest the MongoDB data into SingleStore.

First we'll create a link between the MongoDB Atlas hosts that hold our data and SingleStore.

We'll replace `<password>` with the value that we saved earlier from MongoDB Atlas. We'll also need to replace the values for `<primary>`, `<secondary>` and `<secondary>` with the full address for each from MongoDB Atlas.

In [33]:
%%sql
DROP DATABASE IF EXISTS mktg_email_demo;
CREATE DATABASE IF NOT EXISTS mktg_email_demo;

1 rows affected.

++
||
++
++

In [34]:
%%sql
USE mktg_email_demo;

CREATE LINK mktg_email_demo.link AS MONGODB
CONFIG '{"mongodb.hosts": "<primary>:27017, <secondary>:27017, <secondary>:27017",
        "collection.include.list": "mktg_email_demo.*",
        "mongodb.ssl.enabled": "true",
        "mongodb.authsource": "admin",
        "mongodb.members.auto.discover": "false"}'
CREDENTIALS '{"mongodb.user": "admin",
            "mongodb.password": "<password>"}';

CREATE TABLES AS INFER PIPELINE AS LOAD DATA LINK mktg_email_demo.link '*' FORMAT AVRO;

START ALL PIPELINES;

1 rows affected.

++
||
++
++

### Connect to SingleStore

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [39]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

### Create embeddings for our stages

In [40]:
df_list = []
id_counter = 1

for stage in stages:
    embedding, tokens, status = get_embeddings(stage)

    parent = id_counter - 1

    stage_df = pd.DataFrame(
        {
            "id": [id_counter],
            "content": [stage],
            "embedding": [embedding],
            "parent": [parent]
        }
    )

    df_list.append(stage_df)
    
    id_counter += 1

df = pd.concat(df_list, ignore_index = True)

In [41]:
df["embedding"] = df["embedding"].apply(lambda x: np.array(x, dtype = np.float32))

In [42]:
%config SqlMagic.named_parameters = "enabled"

In [43]:
%%sql
USE mktg_email_demo;

DROP TABLE IF EXISTS docs_splits;

CREATE TABLE IF NOT EXISTS docs_splits (
    id INT,
    content TEXT,
    embedding VECTOR(:dimensions),
    parent INT
);

++
||
++
++

In [44]:
df.to_sql(
    "docs_splits",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

10

In [45]:
%%sql
USE mktg_email_demo;

SELECT content, parent
FROM docs_splits;

10 rows affected.

content,parent
generating a tracking code,1
data export,8
funnels,5
conversion tracking,4
adding tracking to your website,2
dashboard customization,9
getting started,0
user segmentation,6
custom event tracking,7
real-time analytics,3


### Search SingleStore for closest result

In [46]:
def search_s2(vector):
    query = """
        SELECT content, parent
        FROM docs_splits
        ORDER BY (embedding <-> :vector) ASC
        LIMIT 1
    """
    with db_connection.connect() as con:
        result = con.execute(text(query), {"vector": str(vector)})
        return result.fetchone()

### Loop through the users in the `customers` table

Limiting to 5 users, we'll loop through each record in our customers table.

In [47]:
limit = 5
emails = []

# Create a connection
with db_connection.connect() as con:
    query = "SELECT _more :> JSON FROM customers LIMIT :limit"
    result = con.execute(text(query), {"limit": limit})

    for customer in result:
        customer_data = customer[0]
        fname, stage = customer_data["first_name"], customer_data["stage"]

        # Retrieve current and next stage embeddings
        this_stage = next((item for item in stages_w_embed if item["stage"] == stage), None)
        next_stage = next((item for item in stages_w_embed if item["stage"] == find_next_stage(stage)), None)

        if not this_stage or not next_stage:
            continue

        # Get content
        cur_content, cur_permalink = search_s2(this_stage["embedding"])
        next_content, next_permalink = search_s2(next_stage["embedding"])

        # Create the system message
        system_message = f"""
        You are a helpful assistant.
        I am Marketer John at Awesome Web Analytics.
        We are similar to the current top web analytics companies.
        We have users that are at various stages in using our product, and we want to send them helpful emails to get them to use our product more.
        Write an email for {fname} who is on stage {stage} of the onboarding process.
        The next stage is {next_stage['stage']}.
        Ensure the email describes the benefits of moving to the next stage, then always share this link: https://github.com/VeryFatBoy/mktg-email-flow/tree/main/docs/{next_content.replace(' ', '-')}.md.
        Limit the email to 1 paragraph.
        End the email with my signature: 'Best Regards, Marketer John.'
        """

        email = chatgpt_generate_email(system_message, fname)
        emails.append(
            {
                "fname": fname,
                "stage": stage,
                "next_stage": next_stage["stage"],
                "email": email,
            }
        )

### Print out the list

Now, let's print out the emails.

In [48]:
for email in emails:
    print(f"First Name: {email['fname']}\n")
    print(f"Stage: {email['stage']}\n")
    print(f"Next Stage: {email['next_stage']}\n")
    print(f"{email['email']}\n")
    print("_" * 100)

First Name: Brian

Stage: generating a tracking code

Next Stage: adding tracking to your website

Subject: Next Steps to Enhance Your Web Analytics Experience

Hi Brian,

Great job generating your tracking code! The next step is to add this tracking to your website, which will unlock powerful insights into your users' behavior and help you make data-driven decisions. By integrating tracking, you'll be able to see where your visitors are coming from, how they interact with your content, and ultimately optimize your marketing efforts for better results. Ready to take the next step? Check out our guide here: [Adding Tracking to Your Website](https://github.com/VeryFatBoy/mktg-email-flow/tree/main/docs/adding-tracking-to-your-website.md).

Best Regards,  
Marketer John

____________________________________________________________________________________________________
First Name: Caleb

Stage: funnels

Next Stage: user segmentation

Subject: Unlock the Power of User Segmentation

Hi Cale

## Summary

By combining traditional marketing techniques with ChatGPT and powerful database functions, we can automate the writing of effective emails that are contextually important to our users, without the need to spend hours diving into the analytics ourselves.

We can see in this demonstration that we can build a solution using multiple database systems (MongoDB and Pinecone), or we can do this with SingleStore, a powerful database system that combines traditional Relational and NoSQL options with Vectors, enabling us to write our queries in familiar SQL syntax.